# Notebook - Invoke SageMaker Inference Endpoint

**NOTICE: Before running this notebook, please track and wait for the run of the pipeline created by the accompanied notebook `kfp_with_sagemaker_training_s3.ipynb` to complete. Once it completes, use
this notebook to involke a SageMaker Endpoint and submit a sample image for inference** 


Before running this notebok, 
- Make sure that a sample image is uploaded to the notebook instance (below `1000_dog.png` is put in the folder where this notebook is located).
- Update SageMaker Endpoint name below (`Endpoint-cnn-dist-job-2023-05-04-05-47-05-124` for example). There are two ways to retrieve the Endpoint name: 
    1. In the left pane of SageMaker Console, go to `Inference > Endpoint`
    2. Go to the graph visualization of the run of the pipeline, click the component for `SageMaker - Deploy Model`, and go to `Input/Output > endpoint_name`

## Preparation

In [ ]:
import json
import boto3
import numpy as np


In [ ]:
# create a boto3 client for SageMaker
client = boto3.client('runtime.sagemaker')


In [ ]:
# load a sample image
file_name = '1000_dog.png'
with open(file_name, 'rb') as f:
    payload = f.read()


In [ ]:
# map between label and index for CIFAR-10 (index 1 corresponds to 'automobile')
labels = [
    'airplane',
    'automobile',
    'bird',
    'cat',
    'deer',
    'dog',
    'frog',
    'horse',
    'ship',
    'truck'
]


## Invole SageMaker Endpoint for inference

In [ ]:
# NOTE: ensure you update the endpoint name
endpoint_name = 'Endpoint-cnn-dist-job-2023-05-04-05-47-05-124'

response = client.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType='application/x-image', 
    Body=payload
)


In [ ]:
# show the prediction result

pred_raw = json.loads(response['Body'].read().decode())
pred_scores = pred_raw['score'][0]
pred_index = np.argmax(pred_scores)
pred_label = labels[pred_index]

print(f"prediction result: index {pred_index}, label {pred_label}")


### Delete Endpoint

In [ ]:
# # delete endpoint
# sagemaker_client = boto3.client('sagemaker')
# sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
